# Milestone4 - DSC540 (Veera Reddy Koppula)

Perform at least 5 data transformation and/or cleansing steps to your API data. The below examples are not required - they are just potential transformations you could do. If your data doesn't work for these scenarios, complete different transformations. You can do the same transformation multiple times if you needed to clean your data. The goal is a clean dataset at the end of the milestone.

Replace Headers

Format data into a more readable format

Identify outliers and bad data

Find duplicates

Fix casing or inconsistent values

Conduct Fuzzy Matching

In [1]:
#Importing needed libraries
import urllib.request, urllib.parse, urllib.error
import json
import requests
import ssl
import re
import pandas as pd
import io
import numpy as np

In [2]:
#Resetting SSL context to avoid errors in accessing https certs
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
#Retrieving API Key for calling the COVID API
with open('APIkeys.json') as f:
    keys = json.load(f)
    covidapi = keys['COVIDapi']

In [4]:
#Forming URL parameters
keyurl = 'https://api.covidactnow.org/v2/states.timeseries.csv?'
apikey = 'apiKey='+covidapi

In [5]:
#Calling the URL to retrieve CSV data and saving it into a Pandas Data frame
url = keyurl +apikey
print('Retrieving the data from... ')
print(url)
uh = urllib.request.urlopen(url)
data = uh.read()
rawData = pd.read_csv(io.StringIO(data.decode('utf-8')))

Retrieving the data from... 
https://api.covidactnow.org/v2/states.timeseries.csv?apiKey=3d4d39a3bf274ed9bfef33842b6da7ed


In [6]:
#Valdiating successful load
rawData.head()

,date,country,state,county,fips,lat,long,locationId,actuals.cases,actuals.deaths,...,unused3,unused4,metrics.icuCapacityRatio,riskLevels.overall,metrics.vaccinationsInitiatedRatio,metrics.vaccinationsCompletedRatio,actuals.newDeaths,actuals.vaccinesAdministered,riskLevels.caseDensity,cdcTransmissionLevel
0,2020-03-01,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0
1,2020-03-02,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0
2,2020-03-03,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0
3,2020-03-04,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0
4,2020-03-05,US,AK,NaN,2,NaN,NaN,iso1:us#iso2:us-ak,NaN,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0


In [7]:
#Validating size of dataframe
rawData.shape

(32816, 41)

In [8]:
for col in rawData.columns:
    print(col)

date
country
state
county
fips
lat
long
locationId
actuals.cases
actuals.deaths
actuals.positiveTests
actuals.negativeTests
actuals.contactTracers
actuals.hospitalBeds.capacity
actuals.hospitalBeds.currentUsageTotal
actuals.hospitalBeds.currentUsageCovid
unused1
actuals.icuBeds.capacity
actuals.icuBeds.currentUsageTotal
actuals.icuBeds.currentUsageCovid
unused2
actuals.newCases
actuals.vaccinesDistributed
actuals.vaccinationsInitiated
actuals.vaccinationsCompleted
metrics.testPositivityRatio
metrics.testPositivityRatioDetails
metrics.caseDensity
metrics.contactTracerCapacityRatio
metrics.infectionRate
metrics.infectionRateCI90
unused3
unused4
metrics.icuCapacityRatio
riskLevels.overall
metrics.vaccinationsInitiatedRatio
metrics.vaccinationsCompletedRatio
actuals.newDeaths
actuals.vaccinesAdministered
riskLevels.caseDensity
cdcTransmissionLevel


**Step1 - Replace Headers**

In [9]:
workingData=rawData.rename(columns={"actuals.cases": "TotalCases",
"actuals.deaths": "TotalDeaths",
"actuals.positiveTests": "TotalPositiveTests",
"actuals.negativeTests": "TotalNegativeTests",
"actuals.contactTracers": "NumberOfContactTracers",
"actuals.hospitalBeds.capacity": "HospitalBedsCapacity",
"actuals.hospitalBeds.currentUsageTotal": "BedsUsageTotal",
"actuals.hospitalBeds.currentUsageCovid": "BedsUsagebyCovid",
"actuals.icuBeds.capacity": "ICUBedsCapacity",
"actuals.icuBeds.currentUsageTotal": "ICUBedsUsageTotal",
"actuals.icuBeds.currentUsageCovid": "ICUBedsUsagebyCovid",
"actuals.newCases": "NewCases",
"actuals.vaccinesDistributed": "VaccinesDistributed",
"actuals.vaccinationsInitiated": "VaccinationStarted",
"actuals.vaccinationsCompleted": "VaccinationCompleted",
"metrics.testPositivityRatio": "CovidTestPositivityRatio",
"metrics.caseDensity": "CaseDensity",
"metrics.contactTracerCapacityRatio": "ContactTracerCapacityRatio",
"metrics.infectionRate": "InfectionRate",
"metrics.infectionRateCI90": "InfactionRate90Days",
"metrics.icuCapacityRatio": "ICUCapacityRatio",
"riskLevels.overall": "OverallRiskLevels",
"metrics.vaccinationsInitiatedRatio": "VaccinationInitiationRatio",
"metrics.vaccinationsCompletedRatio": "VacconationCompletionRatio",
"actuals.newDeaths": "NewDeaths",
"actuals.vaccinesAdministered": "TotalVaccinesAdministered"})

In [10]:
for col in workingData.columns:
    print(col)

date
country
state
county
fips
lat
long
locationId
TotalCases
TotalDeaths
TotalPositiveTests
TotalNegativeTests
NumberOfContactTracers
HospitalBedsCapacity
BedsUsageTotal
BedsUsagebyCovid
unused1
ICUBedsCapacity
ICUBedsUsageTotal
ICUBedsUsagebyCovid
unused2
NewCases
VaccinesDistributed
VaccinationStarted
VaccinationCompleted
CovidTestPositivityRatio
metrics.testPositivityRatioDetails
CaseDensity
ContactTracerCapacityRatio
InfectionRate
InfactionRate90Days
unused3
unused4
ICUCapacityRatio
OverallRiskLevels
VaccinationInitiationRatio
VacconationCompletionRatio
NewDeaths
TotalVaccinesAdministered
riskLevels.caseDensity
cdcTransmissionLevel


**Step2 - Format data into a more readable format**

In [11]:
#Dropping columns that are not needed to reduce the size of the dataset
workingData=workingData.drop(columns=['country', 'county','lat','long','locationId','unused1','unused2','unused3','unused4','metrics.testPositivityRatioDetails','riskLevels.caseDensity'])

In [12]:
workingData.shape

(32816, 30)

In [13]:
workingData.head()

,date,state,fips,TotalCases,TotalDeaths,TotalPositiveTests,TotalNegativeTests,NumberOfContactTracers,HospitalBedsCapacity,BedsUsageTotal,...,ContactTracerCapacityRatio,InfectionRate,InfactionRate90Days,ICUCapacityRatio,OverallRiskLevels,VaccinationInitiationRatio,VacconationCompletionRatio,NewDeaths,TotalVaccinesAdministered,cdcTransmissionLevel
0,2020-03-01,AK,2,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
1,2020-03-02,AK,2,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
2,2020-03-03,AK,2,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
3,2020-03-04,AK,2,NaN,NaN,NaN,18.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
4,2020-03-05,AK,2,NaN,NaN,NaN,27.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0


**Step3 - Find duplicates**

In [14]:
workingData.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
32811    False
32812    False
32813    False
32814    False
32815    False
Length: 32816, dtype: bool

none of the data elements are duplicated as expected, as they are pegged to individual dates

**Step4-Fix casing or inconsistent values**

In [15]:
#Replacing empty cells with NaN and then replcing by 0 for readability
workingData = workingData.replace(r'^\s*$', np.NaN, regex=True)
workingData.fillna(0, inplace=True)

Ideally there should not be any Empty as the data was loaded from CSV using utf-8 format

In [16]:
workingData.dtypes

date                           object
state                          object
fips                            int64
TotalCases                    float64
TotalDeaths                   float64
TotalPositiveTests            float64
TotalNegativeTests            float64
NumberOfContactTracers        float64
HospitalBedsCapacity          float64
BedsUsageTotal                float64
BedsUsagebyCovid              float64
ICUBedsCapacity               float64
ICUBedsUsageTotal             float64
ICUBedsUsagebyCovid           float64
NewCases                      float64
VaccinesDistributed           float64
VaccinationStarted            float64
VaccinationCompleted          float64
CovidTestPositivityRatio      float64
CaseDensity                   float64
ContactTracerCapacityRatio    float64
InfectionRate                 float64
InfactionRate90Days           float64
ICUCapacityRatio              float64
OverallRiskLevels               int64
VaccinationInitiationRatio    float64
VacconationC

**Step5 -coverting columns to appropriate types to enable the next project milestone for merge**

In [17]:
workingData['date'] = pd.to_datetime(workingData['date'])
workingData['TotalCases']=workingData['TotalCases'].astype(int,errors='raise')
workingData['TotalDeaths']=workingData['TotalDeaths'].astype(int,errors='raise')
workingData['TotalPositiveTests']=workingData['TotalPositiveTests'].astype(int,errors='raise')
workingData['TotalNegativeTests']=workingData['TotalNegativeTests'].astype(int,errors='raise')
workingData['NumberOfContactTracers']=workingData['NumberOfContactTracers'].astype(int,errors='raise')
workingData['HospitalBedsCapacity']=workingData['HospitalBedsCapacity'].astype(int,errors='raise')
workingData['BedsUsageTotal']=workingData['BedsUsageTotal'].astype(int,errors='raise')
workingData['BedsUsagebyCovid']=workingData['BedsUsagebyCovid'].astype(int,errors='raise')
workingData['ICUBedsCapacity']=workingData['ICUBedsCapacity'].astype(int,errors='raise')
workingData['ICUBedsUsageTotal']=workingData['ICUBedsUsageTotal'].astype(int,errors='raise')
workingData['ICUBedsUsagebyCovid']=workingData['ICUBedsUsagebyCovid'].astype(int,errors='raise')
workingData['NewCases']=workingData['NewCases'].astype(int,errors='raise')
workingData['VaccinesDistributed']=workingData['VaccinesDistributed'].astype(int,errors='raise')
workingData['VaccinationStarted']=workingData['VaccinationStarted'].astype(int,errors='raise')
workingData['VaccinationCompleted']=workingData['VaccinationCompleted'].astype(int,errors='raise')
workingData['CaseDensity']=workingData['CaseDensity'].astype(int,errors='raise')
workingData['NewDeaths']=workingData['NewDeaths'].astype(int,errors='raise')
workingData['TotalVaccinesAdministered']=workingData['TotalVaccinesAdministered'].astype(int,errors='raise')

In [18]:
workingData.dtypes

date                          datetime64[ns]
state                                 object
fips                                   int64
TotalCases                             int64
TotalDeaths                            int64
TotalPositiveTests                     int64
TotalNegativeTests                     int64
NumberOfContactTracers                 int64
HospitalBedsCapacity                   int64
BedsUsageTotal                         int64
BedsUsagebyCovid                       int64
ICUBedsCapacity                        int64
ICUBedsUsageTotal                      int64
ICUBedsUsagebyCovid                    int64
NewCases                               int64
VaccinesDistributed                    int64
VaccinationStarted                     int64
VaccinationCompleted                   int64
CovidTestPositivityRatio             float64
CaseDensity                            int64
ContactTracerCapacityRatio           float64
InfectionRate                        float64
InfactionR

In [19]:
#Storing data to a new dataframe for merge operation in Milestone5
CleanData = workingData

In [21]:
CleanData.head()

,date,state,fips,TotalCases,TotalDeaths,TotalPositiveTests,TotalNegativeTests,NumberOfContactTracers,HospitalBedsCapacity,BedsUsageTotal,...,ContactTracerCapacityRatio,InfectionRate,InfactionRate90Days,ICUCapacityRatio,OverallRiskLevels,VaccinationInitiationRatio,VacconationCompletionRatio,NewDeaths,TotalVaccinesAdministered,cdcTransmissionLevel
0,2020-03-01,AK,2,0,0,0,4,0,0,0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
1,2020-03-02,AK,2,0,0,0,4,0,0,0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
2,2020-03-03,AK,2,0,0,0,6,0,0,0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
3,2020-03-04,AK,2,0,0,0,18,0,0,0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
4,2020-03-05,AK,2,0,0,0,27,0,0,0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
